# Aruco detectors

## Pointgrey

In [61]:
import PySpin
import cv2
import tkinter as tk
from tkinter import ttk, filedialog
import threading
import numpy as np
import math

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Camera Acquisition App")

        # Variables to store camera settings
        self.fps = tk.IntVar(value=30)
        self.exposure = tk.IntVar(value=5000)
        self.output_filename = tk.StringVar(value='output')

        # Create GUI elements
        self.create_widgets()

        # ArUco detector
        self.detector = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)

        # VideoWriter and CSV file variables
        self.out = None
        self.csv_file = None
        self.stop_acquisition = threading.Event()  # Event to signal stop of acquisition

    def create_widgets(self):
        # Frame for camera settings
        settings_frame = ttk.LabelFrame(self.root, text="Camera Settings")
        settings_frame.grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)

        # FPS Entry
        ttk.Label(settings_frame, text="FPS:").grid(row=0, column=0, sticky=tk.W)
        fps_entry = ttk.Entry(settings_frame, textvariable=self.fps)
        fps_entry.grid(row=0, column=1, padx=5, pady=5)

        # Exposure Entry
        ttk.Label(settings_frame, text="Exposure (us):").grid(row=1, column=0, sticky=tk.W)
        exposure_entry = ttk.Entry(settings_frame, textvariable=self.exposure)
        exposure_entry.grid(row=1, column=1, padx=5, pady=5)

        # Output Filename Entry
        ttk.Label(settings_frame, text="Output Filename:").grid(row=2, column=0, sticky=tk.W)
        output_entry = ttk.Entry(settings_frame, textvariable=self.output_filename)
        output_entry.grid(row=2, column=1, padx=5, pady=5)

        # Start Button
        start_button = ttk.Button(self.root, text="Start Acquisition", command=self.start_acquisition)
        start_button.grid(row=1, column=0, padx=10, pady=10)

        # Stop Button
        stop_button = ttk.Button(self.root, text="Stop Acquisition", command=self.stop_acquisition_process)
        stop_button.grid(row=1, column=1, padx=10, pady=10)

    def browse_output_file(self):
        filename = filedialog.asksaveasfilename(defaultextension=".avi", filetypes=[("AVI files", "*.avi")])
        if filename:
            self.output_filename.set(filename)

    def configure_camera(self, cam):
        nodemap = cam.GetNodeMap()
        
        # Set acquisition mode to continuous
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())
        
        # Set pixel format to Mono8 (8-bit grayscale)
        node_pixel_format = PySpin.CEnumerationPtr(nodemap.GetNode('PixelFormat'))
        node_pixel_format_mono8 = node_pixel_format.GetEntryByName('Mono8')
        node_pixel_format.SetIntValue(node_pixel_format_mono8.GetValue())
        
        # Set maximum width
        node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
        max_width = node_width.GetMax()
        node_width.SetValue(max_width)
        
        # Set maximum height
        node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
        max_height = node_height.GetMax()
        node_height.SetValue(max_height)
        
        # Enable frame rate control
        node_acquisition_frame_rate_enable = PySpin.CBooleanPtr(nodemap.GetNode('AcquisitionFrameRateEnable'))
        node_acquisition_frame_rate_enable.SetValue(True)

        # Set frame rate
        node_frame_rate = PySpin.CFloatPtr(nodemap.GetNode('AcquisitionFrameRate'))
        node_frame_rate.SetValue(self.fps.get())
        
        # Turn off auto exposure
        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
        node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
        
        # Turn off auto gain
        node_gain_auto = PySpin.CEnumerationPtr(nodemap.GetNode('GainAuto'))
        node_gain_auto_off = node_gain_auto.GetEntryByName('Off')
        node_gain_auto.SetIntValue(node_gain_auto_off.GetValue())
        
        # Set exposure time (in microseconds)
        node_exposure_time = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        node_exposure_time.SetValue(self.exposure.get())
        
        # Set gain (if needed)
        node_gain = PySpin.CFloatPtr(nodemap.GetNode('Gain'))
        node_gain.SetValue(10.0)  # Example: set gain to 10 dB

    def acquire_video(self, cam):
        # Create a VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        frame_width = int(cam.Width())
        frame_height = int(cam.Height())
        self.out = cv2.VideoWriter(f'{self.output_filename.get()}.avi', fourcc, self.fps.get(), (frame_width, frame_height), isColor=False)

        try:
            # Start the acquisition
            cam.BeginAcquisition()

            while not self.stop_acquisition.is_set():
                # Retrieve next image
                image_result = cam.GetNextImage()

                if image_result.IsIncomplete():
                    print('Image incomplete: ', image_result.GetImageStatus())
                else:
                    # Convert image to numpy array (Grayscale)
                    frame = image_result.GetNDArray()

                    # Process frame for ArUco detection
                    self.detect_aruco(frame)

                # Release image
                image_result.Release()

                # Check for 'Esc' key press to exit (optional)
                if cv2.waitKey(1) == 27:  # Esc key
                    break

        finally:
            # End acquisition
            cam.EndAcquisition()

            # Release VideoWriter and destroy OpenCV window
            self.out.release()
            cv2.destroyAllWindows()

    def rotation_matrix_to_euler_angles(R):
        """Convert a rotation matrix to Euler angles (roll, pitch, yaw)"""
        sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
        singular = sy < 1e-6
        if not singular:
            x = math.atan2(R[2, 1], R[2, 2])
            y = math.atan2(-R[2, 0], sy)
            z = math.atan2(R[1, 0], R[0, 0])
        else:
            x = math.atan2(-R[1, 2], R[1, 1])
            y = math.atan2(-R[2, 0], sy)
            z = 0
        return np.degrees(np.array([x, y, z]))  # Convert to degrees

    def detect_aruco(self, frame):
        # f_x = (focal_length_mm * image_width) / sensor_width_mm
        f_x = (13.5*1280)/6.14  # Focal length in pixels (x-axis) 
        f_y = (13.5*1024)/4.92 # Focal length in pixels (y-axis)

        # Define camera parameters (these should be obtained from camera calibration)
        camera_matrix = np.array([[f_x, 0, frame.shape[1] / 2],
                                [0, f_y, frame.shape[0] / 2],
                                [0, 0, 1]], dtype=np.float32)  # Example values, adjust accordingly
        dist_coeffs = np.zeros((5, 1))  # Assuming no lens distortion

        # Detect ArUco markers
        corners, ids, _ = cv2.aruco.detectMarkers(frame, self.detector)

        if ids is not None and len(ids) > 0:
            # Draw detected markers on the frame
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids, borderColor=(255, 255, 255))

            # Estimate pose of each marker to get tvecs (translation vectors)
            marker_length = 0.05  # Example marker length in meters, adjust accordingly
            rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

            for i in range(len(ids)):
                # Get translation vector (position)
                x, y, z = tvecs[i][0]
                position_text = f"Pos: ({x:.2f}, {y:.2f}, {z:.2f}) meters"
                cv2.putText(frame, position_text, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

                # Get rotation matrix from rotation vector
                R, _ = cv2.Rodrigues(rvecs[i])

                # Convert rotation matrix to Euler angles
                roll, pitch, yaw = rotation_matrix_to_euler_angles(R)
                rotation_text = f"Rot: ({roll:.2f}, {pitch:.2f}, {yaw:.2f}) deg"
                cv2.putText(frame, rotation_text, (10, 60 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Display the frame
        cv2.imshow("ArUco Detection", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC key to exit
            self.stop_acquisition_process()
            cv2.destroyAllWindows()


    def start_acquisition(self):
        # Start acquisition in a separate thread
        self.stop_acquisition.clear()  # Clear the stop event flag
        acquisition_thread = threading.Thread(target=self.run_acquisition)
        acquisition_thread.start()

    def run_acquisition(self):
        system = PySpin.System.GetInstance()
        cam_list = system.GetCameras()

        try:
            if cam_list.GetSize() == 0:
                raise ValueError("No cameras found!")

            # Assume one camera for simplicity
            cam = cam_list.GetByIndex(0)

            # Initialize camera
            cam.Init()

            # Configure camera settings
            self.configure_camera(cam)

            # Acquire video until stop event is set
            self.acquire_video(cam)

            # Deinitialize camera
            cam.DeInit()

        except PySpin.SpinnakerException as ex:
            print('Error: %s' % ex)
            exit_code = 1

        finally:
            # Release camera and system resources
            del cam
            cam_list.Clear()
            system.ReleaseInstance()

    def stop_acquisition_process(self):
        # Set the stop event flag to stop acquisition
        self.stop_acquisition.set()

if __name__ == '__main__':
    root = tk.Tk()
    app = CameraApp(root)
    root.mainloop()

## Webcam

In [59]:
import cv2
import numpy as np
import math
import threading

# Load the ArUco dictionary and parameters
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# Open webcam
cap = cv2.VideoCapture(0)

# Define camera matrix and distortion coefficients (dummy values for demonstration)
# These values should ideally be obtained from camera calibration
camera_matrix = np.array([[1000, 0, 320],
                          [0, 1000, 240],
                          [0, 0, 1]], dtype=np.float32)
distortion_coefficients = np.zeros((5, 1), dtype=np.float32)

def rotation_matrix_to_euler_angles(R):
    """Convert a rotation matrix to Euler angles (roll, pitch, yaw)"""
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.degrees(np.array([x, y, z]))  # Convert to degrees

def capture_and_process():
    global latest_frame
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect ArUco markers in the image
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        # If markers are detected
        if len(corners) > 0:
            # Draw detected markers (without axes)
            frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Estimate pose of the markers
            rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, distortion_coefficients)

            # Loop over each detected marker
            for i in range(len(ids)):
                # Get translation vector (position)
                x, y, z = tvecs[i][0]
                position_text = f"Pos: ({x:.2f}, {y:.2f}, {z:.2f}) meters"
                cv2.putText(frame, position_text, (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

                # Get rotation matrix from rotation vector
                R, _ = cv2.Rodrigues(rvecs[i])

                # Convert rotation matrix to Euler angles
                roll, pitch, yaw = rotation_matrix_to_euler_angles(R)
                rotation_text = f"Rot: ({roll:.2f}, {pitch:.2f}, {yaw:.2f}) deg"
                cv2.putText(frame, rotation_text, (10, 60 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Store the latest frame for display
        latest_frame = frame

def display_frame():
    while True:
        if latest_frame is not None:
            # Display the latest frame with overlaid text
            cv2.imshow('Frame', latest_frame)

        # Exit loop when 'Esc' key is pressed
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Release the capture and close the window
    cap.release()
    cv2.destroyAllWindows()

# Shared variable for the latest frame
latest_frame = None

# Create and start threads
capture_thread = threading.Thread(target=capture_and_process)
display_thread = threading.Thread(target=display_frame)

capture_thread.start()
display_thread.start()

# Wait for threads to finish
capture_thread.join()
display_thread.join()
